### Test shift in predictions & accuracy in population model
- For each bird, we create a dataset of n_trials * n_neurons, where trials are the total number of spikes during that trial
- we train a 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
# set to be lower priority
#os.nice(1) 

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas  as pd
from cdcp.paths import DATA_DIR, ensure_dir, FIGURE_DIR
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pathlib2 import Path
from tqdm.autonotebook import tqdm
pd.set_option('display.max_columns', 500)
from joblib import Parallel, delayed

In [6]:
from cdcp.spiketrain_analysis.spiketrain_utils import (
    bin_interp_points,
)

In [7]:
from datetime import datetime

In [8]:
from cdcp.spiketrain_analysis.spiketrain_utils import (
    get_spike_train_vector
)

In [9]:
from cdcp.spiketrain_analysis.unit_stats.cued_similarity_and_spikevector_differences import (
    compute_cued_average_sm_rv_differences
)

In [10]:
from cdcp.paths import DATA_PATH_MAGPI, FIGURE_DIR, ensure_dir
from cdcp.general import save_fig
SAVE_FIGS = True

In [11]:
results_df = pd.read_pickle(DATA_DIR / "population_analyses_with_passive2"/'results_df.pickle')

In [12]:
len(results_df)

29

In [13]:
results_df[:3]

,bird,morph,acc_train,acc_held_out_valid,acc_held_out_invalid,n_train,n_invalid,cm_valid,cm_invalid
0,B1597,AF,0.547657,0.281187,0.273413,11929,1591,"[[103, 111, 55, 37, 10, 12, 3, 2, 0, 1, 2, 0, ...","[[26, 22, 12, 7, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
1,B1597,BE,0.399735,0.183005,0.164223,12078,1705,"[[74, 83, 46, 71, 14, 10, 1, 23, 3, 4, 1, 2, 3...","[[15, 6, 9, 12, 8, 4, 1, 8, 3, 0, 0, 0, 0, 0, ..."
2,B1597,AE,0.521908,0.346580,0.288967,38662,1713,"[[186, 112, 77, 2, 12, 6, 0, 1, 2, 3, 4, 0, 3,...","[[16, 19, 17, 2, 4, 2, 0, 1, 1, 1, 0, 0, 0, 0,..."


In [14]:
def confusion_matrix_to_labels(confusion_matrix):
    x_true = []
    x_predicted = []
    
    for true_label in range(len(confusion_matrix)):
        for predicted_label in range(len(confusion_matrix[true_label])):
            count = confusion_matrix[true_label][predicted_label]
            x_true.extend([true_label] * count)
            x_predicted.extend([predicted_label] * count)
            
    return np.array(x_true), np.array(x_predicted)

### Test whether prediction accuracy is improved for cued stimulus

In [15]:
correctness_dfs = []
for idx, row in tqdm(results_df.iterrows(), total=len(results_df)):
    x_true_valid, x_predicted_valid = confusion_matrix_to_labels(row.cm_valid)
    x_true_invalid, x_predicted_invalid = confusion_matrix_to_labels(row.cm_invalid)

    correct_df = pd.DataFrame({
        'correct':np.concatenate([x_true_valid == x_predicted_valid, x_true_invalid == x_predicted_invalid]),
        'valid':np.concatenate([np.repeat(True, len(x_true_valid)), np.repeat(False, len(x_true_invalid))]),
    })
    correct_df['population'] = row.bird + '_' + row.morph
    correctness_dfs.append(correct_df)
prediction_by_cue_df = pd.concat(correctness_dfs)

  0%|          | 0/29 [00:00<?, ?it/s]

In [16]:
df = prediction_by_cue_df
df['valid'] = df['valid'].astype(float)
df['correct'] = df['correct'].astype('category')

In [17]:
len(df)

246992

In [18]:
df[:3]

,correct,valid,population
0,True,1.0,B1597_AF
1,True,1.0,B1597_AF
2,True,1.0,B1597_AF


In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [20]:
# Mixed Linear Model
model = smf.mixedlm("valid ~ correct", df, groups=df["population"], re_formula="1 + correct")
result = model.fit()

/n/groups/datta/tim_sainburg/conda_envs/cdcp_paper/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [21]:
# Summary of the model
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Mixed Linear Model Regression Results
=======================================================================
Model:                 MixedLM     Dependent Variable:     valid       
No. Observations:      246992      Method:                 REML        
No. Groups:            29          Scale:                  0.1659      
Min. group size:       671         Log-Likelihood:         -128667.2358
Max. group size:       21882       Converged:              Yes         
Mean group size:       8517.0                                          
-----------------------------------------------------------------------
                            Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                    0.789    0.004 217.499 0.000  0.782  0.797
correct[T.True]              0.008    0.004   2.139 0.032  0.001  0.015
Group Var                    0.000    0.000                            
Group x correct[T.True] Cov -0.000    0.000                            
correct[T.True] Var          0.000    0.000                            
=======================================================================

"""